# Store Item Demand Forecasting Challenge - kaggle

## Description

##### Problem

This competition is provided as a way to explore different time series techniques on a relatively simple and clean dataset.

You are given 5 years of store-item sales data, and asked to predict 3 months of sales for 50 different items at 10 different stores.

What's the best way to deal with seasonality? Should stores be modeled separately, or can you pool them together? Does deep learning work better than ARIMA? Can either beat xgboost?

This is a great competition to explore different models and improve your skills in forecasting

##### Evaluation
Submissions are evaluated on SMAPE between forecasts and actual values. We define SMAPE = 0 when the actual and predicted values are both 0.

## Packages

In [2]:
import pandas as pd 
import numpy as np 

## EDA

In [3]:
train = pd.read_csv(r'data/train.csv')
test = pd.read_csv(r'data/test.csv')

In [7]:
# Change date to datetime
train['date'] = pd.to_datetime(train['date'])

# View dataframe
train

,date,store,item,sales
0,2013-01-01,1,1,13
1,2013-01-02,1,1,11
2,2013-01-03,1,1,14
3,2013-01-04,1,1,13
4,2013-01-05,1,1,10
...,...,...,...,...
912995,2017-12-27,10,50,63
912996,2017-12-28,10,50,59
912997,2017-12-29,10,50,74
912998,2017-12-30,10,50,62


In [8]:
# Check descriptive stats
train.describe().T

,count,mean,std,min,25%,50%,75%,max
store,913000.0,5.500000,2.872283,1.0,3.0,5.5,8.0,10.0
item,913000.0,25.500000,14.430878,1.0,13.0,25.5,38.0,50.0
sales,913000.0,52.250287,28.801144,0.0,30.0,47.0,70.0,231.0


In [10]:
print(f"The earliest date in the set is {train.date.min()}")
print("---------------------------")
print(f"The latest date in the set is {train.date.max()}")

The earliest date in the set is 2013-01-01 00:00:00
---------------------------
The latest date in the set is 2017-12-31 00:00:00
